In [1]:
import torch
from transformers import BertTokenizer, BertModel

labels = ['asian',
          'atheist',
          'bisexual',
          'black',
          'buddhist',
          'christian',
          'heterosexual',
          'hindu',
          'homosexual_gay_or_lesbian',
          'intellectual_or_learning',
          'jewish',
          'latino',
          'muslim',
          'other_disability',
          'other_gender',
          'other_race_or_ethnicity',
          'other_religion',
          'other_sexual_orientation',
          'physical',
          'psychiatric_or_mental_illness',
          'transgender',
          'white']

In [3]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.2)
        self.linear = torch.nn.Linear(768, 22) # 22 possible output labels

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()

model.load_state_dict(torch.load("model-4.bin", map_location=torch.device('cpu')))

# Use GPU resources if available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("Using CPU")

model.to(device)

Using CPU


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [10]:
raw_text = """Uncivil? I am civil. Like you I condemn all terrorists attacks, but I do not ever point a finger at everyone, painting them all (be they Muslim, or Right Wing) with a broad brush. I condemn the actions taken by individuals or by a specific hate group.

I simply seek link(s) that support statements. Sometimes I change my opinion upon reading btw..."""

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_text = tokenizer.encode_plus(
    raw_text,
    max_length=256,
    truncation=True,
    add_special_tokens=True,
    return_token_type_ids=True,
    padding=True,
    return_attention_mask=True,
    return_tensors='pt',
)
     
input_ids = encoded_text['input_ids'].to(device)
attention_mask = encoded_text['attention_mask'].to(device)
token_type_ids = encoded_text['token_type_ids'].to(device)
output = model(input_ids, attention_mask, token_type_ids)
 # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
output = torch.sigmoid(output).detach().cpu()
# thresholding at 0.5
output = output.flatten().round().numpy()

print(f"Input: {raw_text}")
labelled = False
for idx, p in enumerate(output):
    if p==1:
        labelled = True
        print(f"Label: {labels[idx]}")
if not labelled:
    print("No Labels applicable")

Input: Uncivil? I am civil. Like you I condemn all terrorists attacks, but I do not ever point a finger at everyone, painting them all (be they Muslim, or Right Wing) with a broad brush. I condemn the actions taken by individuals or by a specific hate group.

I simply seek link(s) that support statements. Sometimes I change my opinion upon reading btw...
Label: muslim
